### PHOENIX-2014-T Dataset

In [4]:
rwth_base_path = "../phoenix-2014-T.v3/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px"
train_path = rwth_base_path + "/train"
test_path = rwth_base_path + "/test"
dev_path = rwth_base_path + "/dev"

In [7]:
import os

def get_image_files(directory, extension=".png"):
    """
    Walks through the given directory and returns a list of file paths 
    with the specified extension (default is .png).
    """
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                image_files.append(os.path.join(root, file))
    
    image_files = [path.replace("\\", '/') for path in image_files]
    return image_files

In [6]:
def save_to_txt(file_list, output_file):
    with open(output_file, 'w') as f:
        f.write("\n".join(file_list))

In [ ]:
# train image paths
train_files = get_image_files(train_path)
save_to_txt(train_files, 'train_images.txt')

In [ ]:
# dev image paths
dev_files = get_image_files(dev_path)
save_to_txt(dev_files, 'dev_images.txt')

In [ ]:
# test image paths
test_files = get_image_files(test_path)
save_to_txt(test_files, 'test_images.txt')

### RTMPose Keypoints Extraction

In [1]:
pose = "Models/rtmpose_onnx/rtmw-x_simcc-cocktail13_pt-ucoco_270e-384x288-0949e3a9_20230925/end2end.onnx"
pose_input_size = (288, 384)
det = "Models/yolox_onnx/yolox_x_8xb8-300e_humanart-a39d44ed/end2end.onnx"
det_input_size = (640, 640)

In [ ]:
import cv2
from rtmlib import Wholebody, draw_skeleton

device = 'cpu'  # cpu, cuda, mps
backend = 'onnxruntime'  # opencv, onnxruntime, openvino
openpose_skeleton = False  # True for openpose-style, False for mmpose-style

wholebody = Wholebody(to_openpose=openpose_skeleton,
                      det = det ,
                      det_input_size = det_input_size,
                      pose = pose,
                      pose_input_size = pose_input_size,
                      mode='balanced',  # 'performance', 'lightweight', 'balanced'. Default: 'balanced'
                      backend=backend, device=device
)

In [ ]:
import cv2
import json

def extract_keypoints(image_files, output_file, start_idx=0, end_idx=None, batch_size=500):
    """
    Extract keypoints for a list of images and save them to a JSON file.
    
    Parameters:
    - image_files: List of image file paths.
    - output_file: Path of the JSON file to save the extracted keypoints.
    - start_idx: Starting index for the image files list.
    - end_idx: Ending index for the image files list (None to process all files).
    - batch_size: Save to the JSON file every 'batch_size' images processed.
    """
    if end_idx is None:
        end_idx = len(image_files)
    
    keypoints_data = []
    
    for i in range(start_idx, end_idx):
        img = cv2.imread(image_files[i])
        keypoints, scores = wholebody(img)
        
        data = {
            "path": image_files[i],
            "keypoints": keypoints.tolist(),
            "scores": scores.tolist()
        }
        keypoints_data.append(data)

        # Log progress and save intermediate results in batches
        if i % batch_size == 0:
            print(f"Processed {i} images.")
            with open(output_file, 'w') as json_file:
                json.dump(keypoints_data, json_file)

    # Save the final batch of data
    with open(output_file, 'w') as json_file:
        json.dump(keypoints_data, json_file)

In [ ]:
def read_file_to_list(filename):

  with open(filename, 'r') as file:
    lines = file.readlines()

  lines = [line.rstrip() for line in lines]

  return lines

In [5]:
# Extract train images keypoints
train_files = read_file_to_list("train_images.txt")

train_output_file = "./rwth_train_keypoints.json"
extract_keypoints(train_files, train_output_file)

In [ ]:
# Extract dev images keypoints
dev_files = read_file_to_list("dev_images.txt")

dev_output_file = "./rwth_dev_keypoints.json"
extract_keypoints(dev_files, dev_output_file)

In [ ]:
# Extract test images keypoints
test_files = read_file_to_list("test_images.txt")

test_output_file = "./rwth_test_keypoints.json"
extract_keypoints(test_files, test_output_file)